In [2]:
import numpy as np
import torch
from deepymod_torch.library_functions import library_1D
from deepymod_torch.DeepMod import DeepMod
from deepymod_torch.sparsity import scaling, threshold
from deepymod_torch.utilities import create_deriv_data

import matplotlib.pyplot as plt
plt.style.use('seaborn-notebook')

np.random.seed(42)

%load_ext autoreload
%autoreload 2

#torch.set_default_tensor_type('torch.cuda.FloatTensor')  # enable for gpu.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
data = np.load('data/keller_segel.npy', allow_pickle=True).item()

In [9]:
X = np.transpose((data['t'].flatten(), data['x'].flatten()))
y = np.real(data['u']).reshape((data['u'].size, 1))

In [6]:
number_of_samples = 500

idx = np.random.permutation(y.size)
X_train = create_deriv_data(torch.tensor(X[idx, :][:number_of_samples], dtype=torch.float32, requires_grad=True), 3)
y_train = torch.tensor(y[idx, :][:number_of_samples], dtype=torch.float32)

In [7]:
config = {'input_dim': 2, 'hidden_dim': 20, 'layers': 5, 'output_dim': 1, 'library_function': library_1D, 'library_args':{'poly_order': 2, 'diff_order': 3}}
model = DeepMod(config)

In [6]:
len(model(X_train))

4

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

In [ ]:
%%time
model.train(X_train, y_train, optimizer, 500)

In [35]:
%%time
model.train(X_train, y_train, optimizer, 500, type='mse')

| Iteration | Progress | Time remaining |     Cost |      MSE |      Reg |       L1 |
        500    100.00%               0s   1.49e-04   1.49e-04   0.00e+00   0.00e+00 CPU times: user 9.23 s, sys: 5.95 s, total: 15.2 s
Wall time: 12.4 s


In [13]:
%%time
model.train(X_train, y_train, optimizer, 100, type='deepmod')

| Iteration | Progress | Time remaining |     Cost |      MSE |      Reg |       L1 |


RuntimeError: invalid argument 8: lda should be at least max(1, 0), but have 0 at ../aten/src/TH/generic/THBlas.cpp:363

In [14]:
model.sparsity_mask_list

[tensor([], dtype=torch.int64)]

In [15]:
model.coeff_vector_list[0]

Parameter containing:
tensor([], size=(0, 1), requires_grad=True)

# testing library for higher input:

In [16]:
# Diffusion, tests 2D input

# General imports
import numpy as np
import torch

# DeepMoD stuff
from deepymod_torch.DeepMod import DeepMod
from deepymod_torch.library_functions import library_1D
from deepymod_torch.utilities import create_deriv_data

# Setting cuda
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [20]:
# Loading data
data = np.load('datasets/diffusion_2D.npy', allow_pickle=True).item()
X = np.transpose((data['t'].flatten(), data['x'].flatten(), data['y'].flatten()))
y = np.real(data['u']).reshape((data['u'].size, 1))
number_of_samples = 1000

idx = np.random.permutation(y.size)
X_train = torch.tensor(X[idx, :][:number_of_samples], dtype=torch.float32, requires_grad=True)
y_train = torch.tensor(y[idx, :][:number_of_samples], dtype=torch.float32)

In [21]:
## Running DeepMoD
config = {'input_dim': 3, 'hidden_dim': 20, 'layers': 5, 'output_dim': 1, 'library_function': library_1D, 'library_args':{'poly_order': 1, 'diff_order': 2}}

X_input = create_deriv_data(X_train, config['library_args']['diff_order'])

model = DeepMod(config)

In [25]:
du = model(X_input)[0][1]

In [26]:
du.shape

torch.Size([1000, 2, 3, 1])

In [27]:
dx = du[:, :, 1:, :]

In [36]:
dx[0, :, 0, 0]

tensor([-0.0035, -0.0006], grad_fn=<SelectBackward>)

In [39]:
dx[0:1, :, :, :].reshape(1, -1, 1)

tensor([[[-0.0035],
         [ 0.0006],
         [-0.0006],
         [-0.0002]]], grad_fn=<UnsafeViewBackward>)

In [50]:
dx[:, :, :, 0].reshape(dx.shape[0], -1).shape

torch.Size([1000, 4])

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)
model.train(X_input, y_train, optimizer, 500, type='deepmod')

print(model.sparsity_mask_list, model.coeff_vector_list[0])

# implementing keller segel

In [30]:
# Keller Segel, tests coupled output.

# General imports
import numpy as np
import torch

# DeepMoD stuff
from deepymod_torch.DeepMod import DeepMod
from deepymod_torch.library_functions import library_basic
from deepymod_torch.utilities import create_deriv_data

# Setting cuda
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
# Loading data
data = np.load('data/keller_segel.npy', allow_pickle=True).item()
X = np.transpose((data['t'].flatten(), data['x'].flatten()))
y = np.transpose((data['u'].flatten(), data['v'].flatten()))
number_of_samples = 1000

idx = np.random.permutation(y.shape[0])
X_train = torch.tensor(X[idx, :][:number_of_samples], dtype=torch.float32)
y_train = torch.tensor(y[idx, :][:number_of_samples], dtype=torch.float32)

In [36]:
## Running DeepMoD
config = {'input_dim': 2, 'hidden_dim': 20, 'layers': 5, 'output_dim': 2, 'library_function': library_basic, 'library_args':{'poly_order': 2, 'diff_order': 2}}

X_input = create_deriv_data(X_train, config['library_args']['diff_order'])

model = DeepMod(config)

In [37]:
data = np.load('data/burgers.npy', allow_pickle=True).item()
X = np.transpose((data['t'].flatten(), data['x'].flatten()))
y = np.real(data['u']).reshape((data['u'].size, 1))
number_of_samples = 1000

idx = np.random.permutation(y.size)
X_train = torch.tensor(X[idx, :][:number_of_samples], dtype=torch.float32)
y_train = torch.tensor(y[idx, :][:number_of_samples], dtype=torch.float32)

## Running DeepMoD
config = {'input_dim': 2, 'hidden_dim': 20, 'layers': 5, 'output_dim': 1, 'library_function': library_basic, 'library_args':{'poly_order': 2, 'diff_order': 2}}

X_input = create_deriv_data(X_train, config['library_args']['diff_order'])

model = DeepMod(config)

In [38]:
output = model(X_input)[0]
X, dX = output
poly_order = 2

In [61]:
# Time deriv shit
dt = dX[:, 0, :1, :]
time_deriv_list = torch.unbind(dt, dim=2)

In [40]:
# Polynomial shit
u = torch.ones_like(X)[:, None, :]
for order in torch.arange(1, poly_order+1):
    u = torch.cat((u, u[:, order-1:order, :] * X[:, None, :]), dim=1)
poly_list = torch.unbind(u, dim=2)

In [41]:
poly_list[0].shape

torch.Size([1000, 3])

In [42]:
dx = dX[:, :, 1:, :]

In [43]:
deriv_list = [torch.cat((torch.ones((dx.shape[0], 1)), eq.reshape(dx.shape[0], -1)), dim=1) for eq in torch.unbind(dx, dim=3)]
samples = dx.shape[0]

In [44]:
if len(poly_list) == 1:
    theta = torch.matmul(poly_list[0][:, :, None], deriv_list[0][:, None, :]).reshape(samples, -1) # If we have a single output, we simply calculate and flatten matrix product between polynomials and derivatives to get library
else:
    theta_uv = torch.cat([torch.matmul(u[:, :, None], v[:, None, :]).reshape(samples, -1) for u, v in combinations(poly_list, 2)], 1)  # calculate all unique combinations between polynomials
    theta_dudv = torch.cat([torch.matmul(du[:, :, None], dv[:, None, :]).reshape(samples, -1)[:, 1:] for du, dv in combinations(deriv_list, 2)], 1) # calculate all unique combinations of derivatives
    theta_udu = torch.cat([torch.matmul(u[:, 1:, None], du[:, None, 1:]).reshape(samples, -1) for u, du in product(poly_list, deriv_list)], 1)  # calculate all unique products of polynomials and derivatives
    theta = torch.cat([theta_uv, theta_dudv, theta_udu], dim=1)


In [45]:
theta.shape

torch.Size([1000, 9])

In [48]:
torch.sum(theta[:, 0] - 1)

tensor(0., grad_fn=<SumBackward0>)

In [52]:
torch.allclose(theta[:, 2], dX[:, 1, 1, 0])

True

In [57]:
torch.allclose(theta[:, 6:7], X**2)

True

In [64]:
time_deriv_list[0].shape

torch.Size([1000, 1])